# Sistema Criptográfico NEWHOPE

O NewHope é um sistema criptográfico constitiuido por mecanismos de encapsulamento KEM, denominado NewHope-CPA-KEM e NEwHope-CCA-KEM, baseado no RLWE(ring learning with errors)

Um aparte:  RLWE é um problema computacional constituido por problemas matematicos considerados dificeis de resolver quando nenhuma informação é disponibilizada, mas facilmente resolvidos quando a informação para a construção do problema é cohecido.

Aqui neste trabalho é pedido para implementar as versoes NewHope-CPA-KEM e NewHope-CCA-PKE.
Para tal foi seguido a documentação disponivel para a implementação do sistema criptográfico, que se encotra em 
https://newhopecrypto.org/data/NewHope_2020_04_10.pdf
Como especificado no documento é possivel converter um sistema IND-CPA-PKE para IND-CPA-KEM e IND-CCA-KEM para IND-CCA-PKE, para tal começamos por implementar o NewHope-CPA-PKE, de maneira a fazer a conversão para IND-CPA-KEM, sendo que continua

#### Sample()

Recebe uma seed aleatoria   
Esta seed depois é usada para realizar sampling em multiplos polinomios     
O output é um polinomio Rq (=Zq[X]/(Xn+ 1)) onde todos os coeficientes são distribuidos igualmente por ψn8  


#### GenA()

Recebe a seed aleatoria.  
A fução usa o Shake128 para expandir a seed   
Esta função resultará num polinomio aleatorio que está no dominio NTT  



#### EncodePoly() , DecodePoly()

EncodePoly converte um polinomio para um bytearray   
O DecodePoly realiza a função contraria, onvertendo um bytearray para um polinomio Rq   



#### EncodePk() , DecodePk()

EncodePk Codifica a chave publica para um array de  928 bytes (n= 512) ou 1824 bytes (n= 1024)  
o DecodePk descodifica a chave publica 



#### Encode, DecodeMsg()

Codifica a mensagem para um polinomio 



#### Compress, Decompress()

Realizam compressão e decompressão da mensagem
Através 


#### EncodeC, DecodeC()

A mensagem é codoficada num array de 1088 bytes (n= 512) ou 2176 bytes (n= 1024)  
O Decode faz a função contrária  


#### CPAPKEKeyGen()

Gera a chave publica e privada

Começa por gerar um polinomio a   
Realiza sampling de um polinomio s, e   
Computaciona NTT de e, s  
calcula a * ntt(s)  
soma o resultante dessa multiplicação com ntt(e), sendo b = a * ntt(s) + ntt(e)  
codifica a chave publica e privada  

#### CPAPKEEncrypt()
    
Encripta a mensagem
Descodifica a chave publica, obtendo a seed e b
Gera um polinomio a com a seed recebida
Faz sampling com de um polinomio s, e com uma coin aleatoria
Calcula NTT(s)
Soma o resultante da multiplicação de a * NTT(s) com NTT(e1)
Codifica a mensagem
Calcula a soma da soma da inversa de b * t com NTT(e2) e v, que é a mensagem codificada

retorna o valor anterior comprimido codificado com o valor de u, que é a soma de a * t com NTT(e1)


#### CPAPKEDecrypt()

Desencripta a mensagem

decodifica o cyphertexto para obter u e h

decodifica a chave privada para obter o polinomio s  
descomprime h para obter v, sendo que realzando a inversa de u *s e subtraindo o resultado a v, obtemos a mensagem codificada  
Para obter a mensagem descodifica o resultado obtido

#### CPA_KEM_KEYGEN()

Mesmo que CPAPKEKeyGen()

#### CPA_KEM_ENCAP()
Gera-se uma coin aleatoria de 32 bytes
realiza-se a hash dessa coin com um byte "\x01", de tamanho 64 bytes
divide-se a hash entre K e coin1
encripta-se o valor de K com o valor coin1, utilizando a função CPAPKEEncrypt
retorna o cyphertexto resultante e o valor da hash de K, que terá 32 bytes

#### CPA_KEM_ENCAP()

Realiza-se a desencriptação com CPAPKEDecrypt, da qual resultará K
realiza-se a hash sobre K, de qual resultará ss


#### CCA_KEM_KEYGEN

obtem-se a chave publica e privada atraves de CPAPKEKeyGen
concatenear chave privada, a chave publica, a hash da chave publica e s
retorna o valor anterior e a chave privada

#### CCA_KEM_ENCAP()

recebe a chave publica
gera uma coina aleatoria
realiza a hash de u, byte '\x04' com a coin, tamanho 32 bytes
separa-se a hash entre K ,coin1 e d

realiza-se a encritpação de u, utilizando a função CPAPKEEncrypt
retorna c+d e ss, que é a hash de K mais a hash de c+d


#### CCA_KEM_DECAP()

obtem-se c1, d1 do cyphertexto, e sk ,pk,h e s da chave privada
obtem-se u, usando CPAPKEDecrypt de c

obtem-se K, coin1 e d com a hash de \x08 + u + hash de pk
verifica-se se c1 = CPAPKEEncrypt(pk,u,coin1) e se d1 == d
se sim retorna-se ss, que é a hash de K mais a hash de c+d


De acordo com a conversão standard da Nist, um sistema KEM pode ser adaptado para um PKE, aplicando ao cyphertexto obtido a partir de CCA_KEM_ENCAP um cyphertexto resultante da aplicação do AES_GCM sobre uma mensagem de  texto limpo
#### CCA_PKE_KEYGEN()

mesmo que CCA_KEM_KEYGEN()

#### CCA_PKE_ENCRYPT(pk, msg):

obtem-se c e ss de CCA_KEM_ENCAP
encripta-se a mensagem com o AES GCM, tuilizando o ss como chave de encriptação
retorna-se o iv o cyphertexto de AESGCM e o cyphertexto resultante de CCA_KEM_ENCAP
   

#### CCA_PKE_DECRYPT(iv, ciphertext, tag,c)

obtem-se ss com utilizar CCA_KEM_DECAP sobre c
desencripta-se a mensagem usando a chave ss
   


In [1]:
import os
import hashlib
import sys

N_Hope = 512
Q_Hope = 12289

In [2]:
class NTT(object):
#    
    def __init__(self, n=128):
        if not any([n == t for t in [512,1024]]):
            raise ValueError("improper argument ",n)
        self.n = n  
        self.q = 12289
        
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen(); self.w = w
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)



In [3]:

#multiply polynomial
def multi(lista, listb):
    return [(a * b)  for a,b in zip(lista,listb)]

#add polynomial
def add(lista,listb):
    return [(a + b)   for a,b in zip(lista,listb)]

#subtract polynomial
def subtract(lista,listb):
    return [(a - b) for a,b in zip(lista,listb)]

# int to binary
def int2bin(x):
    return format(x,'0b')

#binary to int
def bin2int(x):
    return int(x,2)

#Bit Reversal of polynomial
def PolyBitRev(poly):
    rev = [0]*N_Hope
    for i in range(N_Hope):
        rev[BitReversal(i)] = poly[i]

    return rev

#bit reversal
def BitRev(v):
    result = 0
    for i in range(int(log(N_Hope,2))):
        result += (((v >> i) & 1) << (int(log(N_Hope,2)) - 1 - i))
    return result

#bit reversal    
def BitReversal(x):
    s = int2bin(x)[::-1]
    reversedval = bin2int(s)
    return reversedval



In [4]:

T = NTT(N_Hope)

#Sample
#
def Sample(seed,nonce):
    r = [0] * N_Hope
    extseed = os.urandom(34)
    extseed = bytearray(extseed)
    extseed[:32] = seed[:32]
    extseed[32] = nonce
     # Gera noise em lblocks de 64 coeficientes 
    for i in range(N_Hope//64):
        extseed[33] = i
        buf = hashlib.shake_256(extseed).digest(int(128))
        for j in range(64):
            a = buf[2 * j]
            b = buf[2 * j + 1]
            r[64*i+j] = bin(a).count("1") + Q_Hope - (bin(b).count("1")%Q_Hope)
    return r



def GenA(seed):
    a = [0] * N_Hope
    extseed = os.urandom(33)
    extseed = bytearray(extseed)
    extseed[:32]= seed[:32]
    
    for i in range(N_Hope//64):
        ctr = 0
        extseed[32] = i
        state = hashlib.shake_128(extseed)
        while ctr < 64:
            buf = state.digest(int(168))
            j = 0
            while j < 164 and  ctr < 64:
                val = int(buf[j])|(int(buf[j+1])<<8)
                if val < 5 * Q_Hope:
                    a[i*64+ctr] = val
                    ctr +=1
                j+=2
    return a



def EncodePoly(s):
    r = [0]* (7*N_Hope//4)
    for i in range(N_Hope//4):
        t0 = s[(4*i)+0] % Q_Hope
        t1 = s[(4*i)+1] % Q_Hope
        t2 = s[(4*i)+2] % Q_Hope
        t3 = s[(4*i)+3] % Q_Hope
        r[7*i+ 0] = int(t0) & 0xff
        r[7*i+ 1] = (int(t0) >> 8)|(int(t1) << 6) & 0xff
        r[7*i+ 2] = (int(t1) >> 2)&0xff
        r[7*i+ 3] = (int(t1) >> 10)|(int(t2) << 4) & 0xff
        r[7*i+ 4] = (int(t2) >> 4) & 0xff
        r[7*i+ 5] = (int(t2) >> 12)|(int(t3) << 2) & 0xff
        r[7*i+ 6] = (int(t3) >> 6) & 0xff
    return r


def DecodePoly(v):
    
    r = [0]*N_Hope
    for i in range(N_Hope//4):
        r[(4*i)+0] =                       v[(7*i)+0] | (((v[(7*i)+1] & 0x3f)<<8))
        r[(4*i)+1] = (v[(7*i)+1] >> 6) | ((v[(7*i)+2] << 2)) | (((v[(7*i)+3] & 0x0f)<<10))
        r[(4*i)+2] = (v[(7*i)+3] >> 4) | ((v[(7*i)+4] << 4)) | (((v[(7*i)+5] & 0x03)<<12))
        r[(4*i)+3] = (v[(7*i)+5] >> 2) | ((v[(7*i)+6] << 6))
    return r



def EncodePK(b,publicseed):
    size = 7*N_Hope//4 +32
    r = [0]* size
    r[:size-32] = EncodePoly(b)
    r[size-32:] = publicseed[:32]
    return r


def decodePK(pk):
    b1 = DecodePoly(pk[:7*N_Hope//4])
    seed = pk[7*N_Hope//4:]
    return b1, seed



def Encode(msg):
    v = [0]*N_Hope
    for i in range(32):
        for j in range(8):
            mask = -((msg[i]>> j)&1)
            v[ 8*i +int(j)+0] = mask & (Q_Hope//2)
            v[ 8*i +j+256] = mask & (Q_Hope//2)
            if n == 1024:
                v[ 8*i +j+0] = mask & (Q_Hope//2)
                v[ 8*i +j+256] = mask & (Q_Hope//2)
    return v


def DecodeMsg(v):
    u = [0] * 32
    for i in range(0,256):
        tn = abs(int(v[i+0] % Q_Hope) - int((Q_Hope-1)/2))
        tn = tn + abs(int(v[i+256] % Q_Hope) - int((Q_Hope-1)/2))
        if n == 1024: 
            tn = tn + abs(int(v[i+512] % Q_Hope) - int((Q_Hope-1)/2))
            tn = tn + abs(int(v[i+768 % Q_Hope]) - int((Q_Hope-1)/2))
            tn = tn - Q_Hope
  
        else:
            tn = tn - Q_Hope//2
        tn = tn >> 15
        u[i >> 3] = u[i >> 3] | -(tn << (i&7)) 
    return u


def Compress(vl):
    k = 0
    t = [0] * 8
    h = [0] * (3*N_Hope//8) 
    for l in range(0, N_Hope//8):
        i = 8 * l
        for j in range(0,8):
            t[j] = vl[i+j] % Q_Hope
            t[j] = int((int((int(t[j]) << 3)) + Q_Hope/2) / Q_Hope) & 0x7 
        h[k+0] = t[0]|((t[1] << 3))|((t[2] << 6))
        h[k+1] = (t[2] >> 2)|((t[3] << 1))|((t[4] << 4))|((t[5] << 7))
        h[k+2] = (t[5] >> 1)|((t[6] << 2))|((t[7] << 5))
        k = k + 3
    return h



def encodeC(u,h):
    c = [0]*((7*N_Hope//4) + (3* N_Hope//8))
    c[:(7*N_Hope//4)] = EncodePoly(u)
    c[7*N_Hope//4:(7*N_Hope//4) + (3* N_Hope//8)] = h
    return c
            

def DecodeC(c):
    u = DecodePoly(c[:7*N_Hope//4])
    h = c[7*N_Hope//4:]
    return u, h


def Decompress(a):
    r = [0]*N_Hope
    k = 0
    for l in range(N_Hope//8):
        i = 8*l
        r[i+0] =  a[k+0] & 7
        r[i+1] = (a[k+0]>>3) & 7
        r[i+2] = (a[k+0]>>6) | ((a[k + 1]<<2)&4)
        r[i+3] = (a[k+1]>>1) & 7
        r[i+4] = (a[k+1]>>4) & 7
        r[i+5] = (a[k+1]>>7) | ((a[k + 2]<<1)&6)
        r[i+6] = (a[k+2]>>2) & 7
        r[i+7] = (a[k+2]>>5)
        k += 3
        
        for j in range(8):
            r[i+j] = (r[i+j]*Q_Hope+4) >> 3
    return r










In [5]:
def CPAPKEKeyGen():
    seed = os.urandom(32)
    z = hashlib.shake_256(b'\x01' + seed).digest(int(64))
    publicseed = z[:32]
    noiseseed = z[32:64]
    a = GenA(publicseed)
    s = PolyBitRev(Sample(noiseseed,0))
    s1 = T.ntt(s)
    e = PolyBitRev(Sample(noiseseed,1))
    e1 = T.ntt(e)
    multias = multi(a,s1)
    b = add(multias,e1)
    pK = EncodePK(b,publicseed)
    sK = EncodePoly(s1)
    return pK,sK

def CPAPKEEncrypt(pk,msg,coin):
    b, publicseed = decodePK(pk)
    a = GenA(publicseed)
    s1 = PolyBitRev(Sample(coin,0))
    e1 = PolyBitRev(Sample(coin,1))
    e2 = Sample(coin,2)
    t = T.ntt(s1)
    tempu = multi(a,t)
    u = add(tempu,T.ntt(e1))
    v = Encode(msg)
    invnttbt = T.ntt_inv(multi(b,t))
    invnttbte2 = add(invnttbt,e2)
    v1 = add(invnttbte2,v)
    h = Compress(v1)
    
    return encodeC(u,h)

def CPAPKEDecrypt(c,sk):
    u,h = DecodeC(c)
    s = DecodePoly(sk)
    v = Decompress(h)
    mult = multi(u,s)
    inv = T.ntt_inv(mult)
    subtr= subtract(v,inv)
    msg = DecodeMsg(subtr)
    return msg

In [6]:
pk, sk = CPAPKEKeyGen()
#print(pk)
#print(sk)
import random
msg = random.sample(range(255), 32)

print("message: ")
#msg = [225, 235, 49, 214, 170, 104, 167, 11, 44, 191, 245, 93, 225, 169, 110, 109, 210, 245, 50, 76, 61, 222, 120, 169, 152, 103, 251, 147, 188, 248, 161, 144]
stuff = CPAPKEEncrypt(pk,msg,os.urandom(32))
print(msg)
#print(stuff)
print("decrypt: \n")
print(CPAPKEDecrypt(stuff,sk))
#c, ss = CPA_KEM_ENCAP(pk)
#print(len(b'\x01'))
#decap = CPA_KEM_DECAP(c,sk)
#print(NTT(KeyGen(),10302,q))

message: 
[168, 251, 95, 245, 85, 105, 70, 31, 51, 172, 213, 90, 27, 241, 189, 134, 181, 10, 153, 220, 0, 225, 178, 116, 140, 129, 92, 62, 93, 8, 177, 176]
decrypt: 

[168, 251, 95, 245, 85, 105, 70, 31, 51, 172, 213, 90, 27, 241, 189, 134, 181, 10, 153, 220, 0, 225, 178, 116, 140, 129, 92, 62, 93, 8, 177, 176]


In [7]:
def CPA_KEM_KEYGEN():
    return CPAPKEKeyGen()

def CPA_KEM_ENCAP(pk):
    coin = os.urandom(32)
    digest = hashlib.shake_256(b'\x01' + coin).digest(int(64))
    K = digest[:32]
    coin1 = digest[32:64]
    c = CPAPKEEncrypt(pk,K,coin1)
    ss = hashlib.shake_256(K).digest(int(32))
    return c, ss

def CPA_KEM_DECAP(c,sk):
    K = CPAPKEDecrypt(c,sk)
    ss = hashlib.shake_256(bytes(K)).digest(int(32))
    return ss
    

In [8]:
pk, sk = CPA_KEM_KEYGEN()
c, ss = CPA_KEM_ENCAP(pk)
#print(len(b'\x01'))
decap = CPA_KEM_DECAP(c,sk)

ss == decap

True

In [9]:


def CCA_KEM_KEYGEN():
    pk, sk = CPAPKEKeyGen()
    
    s = os.urandom(32)
    sk = bytes(sk) + bytes(pk) + hashlib.shake_256(bytes(pk)).digest(int(32))+s
    return(pk,sk)

def CCA_KEM_ENCAP(pk):
    coin = os.urandom(32)
    u = hashlib.shake_256(b'\x04' + coin).digest(int(32))
    digest = hashlib.shake_256(b'\x08'+u+hashlib.shake_256(bytes(pk)).digest(int(32))).digest(int(96))
    K = digest[:32]
    coin1 = digest[32:64]
    d = digest[64:96]
    c = CPAPKEEncrypt(pk,u,coin1)
    pack = b''
    for i in c:
        pack += (int(i)).to_bytes(2, byteorder='big')
    ss = hashlib.shake_256(K + hashlib.shake_256(pack+d).digest(int(32))).digest(int(32))
    return c+list(d), ss

def CCA_KEM_DECAP(c,sk):
    c1 = c[:3*N_Hope//8 + 7*N_Hope//4]
    d1 = bytes(c[3*N_Hope//8 + 7*N_Hope//4:])
    sk1 = list(sk[:(7*N_Hope//4)])
    pk = list(sk[(7*N_Hope//4):(7*N_Hope//4)*2 +32])
    h = sk[(7*N_Hope//4)*2 +32:(7*N_Hope//4)*2 + 32 + 32]
    s = sk[(7*N_Hope//4)*2 +32 +32:(7*N_Hope//4)*2 + 32 + 32 + 32]
    u = bytes(CPAPKEDecrypt(c, sk1))
    digest = hashlib.shake_256(b'\x08'+u+hashlib.shake_256(bytes(pk)).digest(int(32))).digest(int(96))
    K = digest[:32]
    coin1 = digest[32:64]
    d = digest[64:96]
    if c1 == CPAPKEEncrypt(pk,u,coin1) and d1 == d:
        pack = b''
        for i in c1:
            pack += (int(i)).to_bytes(2, byteorder='big')
        return hashlib.shake_256(K + hashlib.shake_256(pack+d).digest(int(32))).digest(int(32))
    else:
        raise TypeError("decryption error,bye")



In [10]:
pk, sk = CCA_KEM_KEYGEN()

#print (sk)
c, ss = CCA_KEM_ENCAP(pk)
ss == CCA_KEM_DECAP(c,sk)



True

In [11]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

def CCA_PKE_KEYGEN():
    return CCA_KEM_KEYGEN()

def CCA_PKE_ENCRYPT(pk, msg):
    c, ss = CCA_KEM_ENCAP(pk)
    iv = os.urandom(12)
    encryptor = Cipher(algorithms.AES(ss),modes.GCM(iv),backend=default_backend()).encryptor()
    ciphertext = encryptor.update(msg) + encryptor.finalize()
    return iv, ciphertext, encryptor.tag,c

def CCA_PKE_DECRYPT(iv, ciphertext, tag,c):
    ss = CCA_KEM_DECAP(c,sk)
    decryptor  = Cipher(algorithms.AES(ss),modes.GCM(iv,tag),backend=default_backend()).decryptor ()
    return decryptor.update(ciphertext) + decryptor.finalize()

ModuleNotFoundError: No module named 'cryptography'

In [ ]:
pk, sk = CCA_PKE_KEYGEN()
iv, ciphertext, tag,c = CCA_PKE_ENCRYPT(pk, b'dddw')
print(CCA_PKE_DECRYPT(iv, ciphertext, tag,c))

## NTRU Prime

Primeiramente, o NTRU Prime oferece varias vantagens de implementação e de auditoria de segurança, alem da escolha do anel NTRU Prime, como por exemplo:
Elimina a possibilidade  de "decryption failures" que comumente aparecem na grande parte dos sistemas de criptografia baseados em lattice. Segundo alguns documentos, foi possivel verificar que algumas versoes do NTRU Prime oferecem uma grande margem de segurança além do nivel de segurança desejado, compensando o possivel progresso na estimativa do custo real dos ataque a lattice.

Para Desenvolver esta implentaçao, usamos como suporte o documento NTRU Prime: round 2 20190330, que nos diz que o NTRU PRIME possui 2 layers. A camanda interna, é Streamlined NTRU Prime Core, uma PKE deterministica perfeitamente correta. Enquanto que a outra layer é uma KEM. Com base no que foi referido acima, esta implementaçao possuirá alguns metodos auxiliares provenientes deste documento.


## GERAÇÃO DE PARAMETROS

De um ponto de vista mais simplorio, o NTRU Prime pode ser visto como uma familia de sistemas de criptografia parametrizados por numero inteiros positivos(variavel p, q e ainda o w) que ainda sao sujeitos algumas restriçoes:
     p é um numero primo  2*p >= 3*w
     q é um numero primo, q >= 16*w+1
     w é inteiro maior que 0

Abreviamos o anel Z[x]/(xp-x-1) como R, o anel (Z/3)[x]/(xp-x-1)como R/3 e o campo (Z/q)[x]/(xp-x-1) como R/q.

Nota:
-Elemento de R é tratado como pequeno se todos os seu coeficientes estiverem em {-1,0,1}.

## PROCESSO DE GERAÇÃO DE CHAVES
Neste processo, primeiro o receptor gera uma chave publica:

    - Gerar um pequeno elemento aleatorio uniforme g ∈ R.
      - Repetir passo, até que g seja invertivel no R/3.
    - Gerar um elemento t-pequeno aleatorio uniforme f ∈ R. (f é diferente de 0, logo invertivel em R/q, pois t≥1).
      - Calcular h = g/(3f) em R/q. (Neste passo, supondo que q é um primo maior que 3, entao 3 é invertivel em R/q)
      - Codificar h como uma sequencia h. A chave publica é h.
      - Guardar os seguintes segredos:
              -f em R e 1/g em R/3.
              
Se q for menor que uma potencia de 2, é possivel compactar facilmente a chave publica misturando elementos adjacentes de Z/q com um limite inferior de p log2 q bits.

## ENCRYPT

Define Inputs = Short e Ciphertext = Rounded
Encrypt mapeia Inputs x PublicKey e CipherTexts, este Algoritmo deterministico, segue o seguinte pseudocodigo

    -Input r ∈ Inputs e h ∈ PublicKeys.
    -Calcular hr ∈ R/q
    -Output Round(hr).
    
## Decrypt

Este algoritmo deterministico, mapeia Ciphertexts x SecretKeys em Inputs, da seguinda maneira:

    -Input ct ∈ Ciphertexts e (f,v)∈ Short x R/3
    -Calcular 3fc ∈ R/q ,-Pode-se multiplicar ct por 3 e depois por f, ou multiplicar c por um 3f pré-computado ou multiplicar c por f e depois por 3.)
    -Ver cada coeficiente de 3fc em R/q como um número inteiro entre -(q - 1)/2 e (q-1)/2,e depois reduce módulo 3, obtendo um polinômio e ∈ R/3.
    -Multiplicar por v no R / 3. Lift ev no R / 3 para um pequeno polinômio r' ∈ R.
    -Output r' se r' tiver weigth w. Se não, output(1,1,....,1,0,0..,0). 

Nota: 
    Preciso ter cuidado para evitar fugas de informaçoes secretas através dos side channels  evitar a implementaçao do weight aqui como um branch.

In [12]:
#Codigo baseado do NtruPrime.sage round2
import hashlib

#Parametros
#x^p -x -1 é irredutivel no polinomio ring(Z/q)[x]
p = 761
q61 = 765
q = 6*q61+1
w = 286

#Apenas para verificar se os parametros estão corretos
def verify(p,q,w):
    print('VERIFICADOR DE PARAMETROS')
    # p numero primo, q numero primo e w inteiro positivo
    #2p>= 3w  -- CHECK
    if p.is_prime and 2*p >= 3*w: print(' p é primo e 2*p >= 3*w')
    # q >= 16w + 1 -- CHECK
    if q.is_prime and q >= 16*w+1: print(' q é primo e q >= 16*w+1')
    #W maior que 0    
    if w > 0: print(' w é > 0')


F3 = GF(3)

Fq = GF(q)
q12 = ZZ((q-1 )/2 )
Zx.<x> = ZZ[]
R.<xp> = Zx.quotient(x^p-x-1)
F3x.<x3> = F3[]
R3.<x3p> = F3x.quotient(x^p-x-1)
Fqx.<xq> = Fq[]
Rq.<xqp> = Fqx.quotient(x^p-x-1)

def ZZ_fromF3(c):
    assert c in F3
    return ZZ(c+Integer(1))-Integer(1)

def ZZ_fromFq(c):
    assert c in Fq
    return ZZ(c+q12)-q12

#------- polynomials mod 3

def R_fromR3(r):
    assert r in R3
    return R([ZZ_fromF3(r[i]) for i in range(p)])

def R3_fromR(r):
    assert r in R
    return R3([r[i] for i in range(p)])
    
# ----- polynomials mod q

def R_fromRq(r):
    assert r in Rq
    return R([ZZ_fromFq(r[i]) for i in range(p)])

def Rq_fromR(r):
    assert r in R
    return Rq([r[i] for i in range(p)])


def Weightw_is(r):
    assert r in R
    w == len([i for i in range(p) if r[i] != 0 ])
    return w == len([i for i in range(p) if r[i] != 0 ])

def Small_is(r):
    assert r in R
    return all(abs(r[i]) <= 1  for i in range(p))

def Short_is(r):
    return Small_is(r) and Weightw_is(r)


# ----- rounded polynomials mod q
# ----- rounded polynomials mod q
#Definir Rounded como o conjunto de elementos,
#onde cada coeficiente de ri is in {−(q − 1)/2, . . . , −6, −3, 0, 3, 6, . . . ,(q − 1)/2} for q ∈ 1 + 3Z
def Rounded_is(r):
    assert r in R
    return (all(r[i]%3  == 0  for i in range(p))
        and all(r[i] >= -q12 for i in range(p))
        and all(r[i] <= q12 for i in range(p)))
#Definir Round : R/q → Rounded 
#if ai ∈ {−(q − 1)/2, . . . ,(q − 1)/2}, a pertence a Rq
def Round(a):
    assert a in Rq
    c = R_fromRq(a)
    r = [3 *round(c[i]/3 ) for i in range(p)]
    assert all(abs(r[i]-c[i]) <= 1  for i in range(p))
    r = R(r)
    assert Rounded_is(r)
    return r

# ----- sorting to generate short polynomial

def Short_fromlist(L): # L is list of p uint32
    L = [L[i]&-2  for i in range(w)] + [(L[i]&-3 )|1  for i in range(w,p)]
    assert all(L[i]%2  == 0  for i in range(w))
    assert all(L[i]%4  == 1  for i in range(w,p))
    L.sort()
    L = [(L[i]%4 )-1  for i in range(p)]
    assert all(abs(L[i]) <= 1  for i in range(p))
    assert sum(abs(L[i]) for i in range(p)) == w
    r = R(L)
    assert Short_is(r)
    return r

def random8(): return randrange(256 )

def urandom32():
    c0 = random8()
    c1 = random8()
    c2 = random8()
    c3 = random8()
    return c0 + 256 *c1 + 65536 *c2 + 16777216 *c3

def Short_random(): # R element with w coeffs +-1
    L = [urandom32() for i in range(p)]
    return Short_fromlist(L)

def randomrange3():
    return ((urandom32() & 0x3fffffff ) * 3 ) >> 30 

def Small_random():
    r = R([randomrange3()-1  for i in range(p)])
    assert Small_is(r)
    return r

In [13]:
#Define PublicKeys = R/q and SecretKeys = Short * R/3.
#Função responsavel por gerar como saída PublicKey x SecretKeys
def KeyGen():
    while (True):
        #Gerar um valor pequeno g pertencente a R
        #Repetir processo até q g seja invertivel em R/3
        g = R([randomrange3()-1  for i in range(p)])
        if R3_fromR(g).is_unit(): break
    #Gerar um uniform random f pertencente Short
    #Privado, f é diferente de 0, logo invertivel em R/q, t>= 1
    f = Short_random()
    #Polinómio h é a informação pública
    #h = g/3f em R/q
    h = Rq_fromR(g)/Rq_fromR(3 *f)
    # supondo que q é um primo > 3, então 3 é inversivel em R/q, logo 3f também é  
    #Output(h,(f,1/g))∈ PublicKeys x SecretKeys.
    return h,(f,1 /R3_fromR(g))
#Define Inputs = Short e Ciphertext = Rounded
#Algoritmo deterministico, Encrypt mapeia Inputs x PublicKey e CipherTexts
def Encrypt(r,h):
    assert Short_is(r)
    #Calcular pkr ∈ R/q 
    assert h in Rq
    #iv = os.urandom(12)
    #Output Round()
    return Round(h*Rq_fromR(r))
#Algoritmo Deterministico, Decrypt mapeia Ciphertexts x SecretKeys em Inputs
def Decrypt(c,k):
    #Input ct ∈ Ciphertexts e (f,v)∈ Short x R/3
    f,v = k
     #Calcular 3fc ∈ R/q - 
    #One can multiply ct by 3 and then by f, or multiply ct by a
    #precomputed 3f, or multiply ct by f and then by 3.)
    #Ver cada coeficiente de 3f c em R / q como um número inteiro entre - (q - 1) / 2 e (q - 1) / 2,
    #e depois reduzir o módulo 3, obtendo um polinômio e ∈ R / 3
    e = R3_fromR(R_fromRq(3 *Rq_fromR(f)*Rq_fromR(c)))
    #Multiplicar por v no R / 3. Lift ev no R / 3 para um pequeno polinômio r' ∈ R.
    r = R_fromR3(e*v)
    #Output r' se r' tiver weight w. se nao output(1,1...,1,0,0)
    if Weightw_is(r): return r
    #Preciso ter cuidado para evitar fugas de informaçoes secretas através dos side channels
    #e evitar a implementaçao do weight aqui com um branch.
    return R([1]*w+[0]*(p-w))

In [14]:
#Verificar os parametros
ver = verify(p,q,w)
#Gerar as chaves
h,j = KeyGen()
#Gerar valor aleatorio pequeno que pertença a R
r = Short_random()
#Ciphertext = Encripta com o valor pequeno e chave publica h
ct = Encrypt(r,h)
#Decrypt - Ciphertext e a chave privado
rr = Decrypt(ct,j)
#print(r)
print(KeyGen())
#Verificar se r  
if r == rr:
    print('True!')

VERIFICADOR DE PARAMETROS
 p é primo e 2*p >= 3*w
 q é primo e q >= 16*w+1
 w é > 0
(1687*xqp^760 + 4324*xqp^759 + 3570*xqp^758 + 3982*xqp^757 + 1213*xqp^756 + 2511*xqp^755 + 578*xqp^754 + 407*xqp^753 + 2339*xqp^752 + 1836*xqp^751 + 4135*xqp^750 + 1910*xqp^749 + 2600*xqp^748 + 43*xqp^747 + 3132*xqp^746 + 1008*xqp^745 + 274*xqp^744 + 721*xqp^743 + 4435*xqp^742 + 2494*xqp^741 + 1136*xqp^740 + 1206*xqp^739 + 163*xqp^738 + 2301*xqp^737 + 2487*xqp^736 + 747*xqp^735 + 2626*xqp^734 + 3629*xqp^733 + 2168*xqp^732 + 2713*xqp^731 + 484*xqp^730 + 968*xqp^729 + 1094*xqp^728 + 2421*xqp^727 + 2294*xqp^726 + 1193*xqp^725 + 1865*xqp^724 + 2943*xqp^723 + 1502*xqp^722 + 2603*xqp^721 + 3724*xqp^720 + 1116*xqp^719 + 2530*xqp^718 + 3450*xqp^717 + 4258*xqp^716 + 2486*xqp^715 + 2561*xqp^714 + 4082*xqp^713 + 110*xqp^712 + 4358*xqp^711 + 2857*xqp^710 + 1163*xqp^709 + 4587*xqp^708 + 2218*xqp^707 + 3111*xqp^706 + 1530*xqp^705 + 1385*xqp^704 + 4507*xqp^703 + 3430*xqp^702 + 356*xqp^701 + 2991*xqp^700 + 3187*xqp^699

# NTRU Prime

Primeiramente, o NTRU Prime oferece varias vantagens de implementação e de auditoria de segurança, alem da escolha do anel NTRU Prime, como por exemplo:
Elimina a possibilidade  de "decryption failures" que comumente aparecem na grande parte dos sistemas de criptografia baseados em lattice. Segundo alguns documentos, foi possivel verificar que algumas versoes do NTRU Prime oferecem uma grande margem de segurança além do nivel de segurança desejado, compensando o possivel progresso na estimativa do custo real dos ataque a lattice.


## GERAÇÃO DE PARAMETROS

De um ponto de vista mais simplorio, o NTRU Prime pode ser visto como uma familia de sistemas de criptografia parametrizados por numero inteiros positivos(variavel p, q e ainda o t) que ainda sao sujeitos algumas restriçoes:
    p é um numero primo
    q é um numero primo
    t é maior ou igual a 1
    p deve ser maior ou igual a 3t
    q deve ser maior ou igual 32t + 1
    e ainda,
    x*p-x-1 é irredutível no anel polinomial (Z/q)[x].

Abreviamos o anel Z[x]/(xp-x-1) como R, o anel (Z/3)[x]/(xp-x-1)como R/3 e o campo (Z/q)[x]/(xp-x-1) como R/q.

Nota:
-Elemento de R é tratado como pequeno se todos os seu coeficientes estiverem em {-1,0,1}.
-Elemento t-small se exatamente 2t do seus coeficientes forem diferentes de zero, ou seja o seu peso de Hamming é igual a 2t.

## PROCESSO DE GERAÇÃO DE CHAVES
Neste processo, primeiro o receptor gera uma chave publica:
      - Gerar um pequeno elemento aleatorio uniforme g ∈ R.
      - Repetir passo, até que g seja invertivel no R/3.
      - Gerar um elemento t-pequeno aleatorio uniforme f ∈ R. (f é diferente de 0, logo invertivel em R/q, pois t≥1).
      - Calcular h = g/(3f) em R/q. (Neste passo, supondo que q é um primo maior que 3, entao 3 é invertivel em R/q)
      - Codificar h como uma sequencia h. A chave publica é h.
      - Guardar os seguintes segredos:
              -f em R e 1/g em R/3.

A codificação das chaves publica como string tambem é um parametro do NTRU Prime. 
Cada elemento do Z/q é codificado como dlog2 qe bits, logo a chave publica é codificada como pdlog2 qe bits.

Se q for menor que uma potencia de 2, é possivel compactar facilmente a chave publica misturando elementos adjacentes de Z/q com um limite inferior de p log2 q bits.

## ENCAPSULAMENTO

Por si só o NTRU Prime é um mecanismo de encapsulamento de chave ou KEM. O que quer dizer, que o remetente recebe uma chave publica como entrada e produz um texto cifrado e uma chave de sessao como saída.

Neste processo o remetente gera um texto cifrado da seguinte maneira:

    -1º Decodificar a chave publica h, obtendo h ∈ R/q.
    -2º Gerar um elemento t-pequeno aleatorio uniforme r ∈ R. 
    -3º Calcular hr ∈ R/q
    -4º Arredondar cada coeficiente de hr, visto como um numero inteiro entre −(q − 1)/2 and (q − 1)/2, isto para o multiplo mais proximo de 3, consequentemente produzindo c ∈ R.
    - (Se q ∈ 1 + 3Z, como em nosso estudo de caso q = 4591, então cada coeficiente de c está em {- (q - 1) / 2,..., −6, −3, 0, 3, 6,..., (q - 1) / 2}. Se q ∈ 2 + 3Z, então cada coeficiente de c está em {- (q + 1) / 2,..., −6, −3, 0, 3, 6,..., (Q + 1) / 2}.)
    
5º Codificar c como uma string c.
6º Hash r, obtendo a parte esquerda C, que é a Chave de confirmaçao e a parte direita K.
Por ultimo o texto cifrado é a concatenaçao C c. E a Chave de sessão é K.


## DECAPSULAÇÃO

Neste processo o recetor decapsula um texto cifrado C c da seguinte maneira:
    
    -1º Decoficar complementar do conjunto c, obtendo c ∈ R. 
    -2º Multiplicar por 3f em R/q.
    -3º Ver cada coeficiente de 3fc em R/q como um inteiro entre -(q-1)/2 e (q-1)/2 e reduzir modulo 3, obtendo assim -um polinomio (e) em R/3.
    -4º Multiplicar por 1/g em R/3
    -5º  Lift e/g in R/3 para um pequeno polinomio r'∈ R
    -6º Calcular c',C',K' a partir de r' como no processo de encapsulamento.
    -7º Verificar se r' for t-pequeno, c' = c e C' = C,
        - Se for, produzir K'.
        - Se não, devolver False.


Se Cc é um texto cifrado legítimo, então c é obtido arredondando os coeficientes de hr para os múltiplos mais próximos de 3;
r' = r é um t-pequeno, c' = c e C' = C, portanto, o decapsulamento gera K' = K, a mesma chave de sessão produzida pelo encapsulamento.


## ANEL

A escolha do sistema criptografico inclui a escolha de um polinomio tendo o coeficiente do termo de maior grau igual a um. Ou melhor de grau-p P ∈ Z[x] e ainda a escolha dum numero inteiro positivo q.

Aqui abreviamos o anel Z[x]/P como R e o anel(Z/q)[x]/P como R/q. 
Segundo a documentaçao, as opçoes de P mencionadas incluem xp-x-1 para p. E as opçoes de q incluem primos inertes q.

## ChAVE PUBLICA

Como já verificado, a chave publica do recetor (h) é um elemento de R/q, que é calculado pela divisao de 2 polinomios. É invertivel no R/q, mas nao possui outra estrutura publica visivel. A alternativa que foi descrita nos varios documentos usados como referencia, afirmam que a alternativa baseia-se na transmissao da chave publica (h) como dois elementos d, hd ∈ R/q, onde d é escolhido como um elemento invertivel aleatorio uniforme de R/q. A este processo chamam "coordenadas projetivas aleatorias", enquanto o simple envio de h seria chamado de "coordenadas afins". Desta forma o recetor salta todas as divisoes na computaçao secreta da chave publica (h).
O recetor calcula h como uma fraçao, e depois multiplica o numerador e dominador po um elemento invertivel e aleatorio de R/q.


In [15]:
#Funções auxiliares retiradas de 
p = 761 
q61 = 765
q = 6*q61+1
t = 143
Zx.<x> = ZZ[]
R.<xp> = Zx.quotient(x^p-x-1)
Fq = GF(q)
Fqx.<xq> = Fq[]
Rq.<xqp> = Fqx.quotient(x^p-x-1)
F3 = GF(3) 
F3x.<x3> = F3[]
R3.<x3p> = F3x.quotient(x^p-x-1)

import hashlib

def hash(s): 
    h = hashlib.sha512()
    h.update(s)
    return h.digest()

def random32(): 
    return randrange(-2^31,2^31)

def random32even(): 
    return random32() & (-2)

def random321mod4(): 
    return (random32() & (-3)) | 1

def randomrange3(): 
    return ((random32() & 0x3fffffff) * 3) >> 30

import itertools

def concat(lists): 
    return list(itertools.chain.from_iterable(lists))

def nicelift(u):  
    return lift(u + q//2) - q//2

def nicemod3(u): # r in {0,1,-1} with u-r in {...,-3,0,3,...}  
    return u - 3*round(u/3)

def int2str(u,bytes):  
    return ''.join(chr((u//256^i)%256) for i in range(bytes))

def str2int(s):  
    return sum(ord(s[i])*256^i for i in range(len(s)))
def seq2str(u,radix,batch,bytes): # radix^batch <= 256^bytes  
    return ''.join(int2str(sum(u[i+t]*radix^t for t in range(batch)),bytes)
                   for i in range(0,len(u),batch))

def str2seq(s,radix,batch,bytes):  
    u = [str2int(s[i:i+bytes]) for i in range(0,len(s),bytes)]  
    return concat([(u[i]//radix^j)%radix for j in range(batch)] for i in range(len(u)))

def encodeZx(m): # assumes coefficients in range {-1,0,1}  
    m = [m[i]+1 for i in range(p)] + [0]*(-p % 4)  
    return seq2str(m,4,4,1)

def decodeZx(mstr):  
    m = str2seq(mstr,4,4,1)  
    return Zx([m[i]-1 for i in range(p)])

def encodeRq(h):  
    h = [q//2 + nicelift(h[i]) for i in range(p)] + [0]*(-p % 5)  
    return seq2str(h,6144,5,8)[:1218]

def decodeRq(hstr):  
    h = str2seq(hstr,6144,5,8)  
    if max(h) >= q: 
        raise Exception("pk out of range")  
    return Rq([h[i]-q//2 for i in range(p)])

def encoderoundedRq(c):  
    c = [q61 + nicelift(c[i]/3) for i in range(p)] + [0]*(-p % 6)  
    return seq2str(c,1536,3,4)[:1015]

def decoderoundedRq(cstr):  
    c = str2seq(cstr,1536,3,4)  
    if max(c) > q61*2: 
        raise Exception("c out of range")  
    return 3*Rq([c[i]-q61 for i in range(p)])

In [16]:
def randomR(): # R element with 2t coeffs +-1  
    L = [random32even() for i in range(2*t)]  
    L += [random321mod4() for i in range(p-2*t)]  
    L.sort()  
    L = [(L[i]%4)-1 for i in range(p)]  
    print (L)
    return Zx(L)

def keygen():
    #Gerar um pequeno elemento aleatorio uniforme g.
    #Repetir até g ser inversivel no R/3
    while True:
        #g privado
        g = Zx([randrange(3)-1 for i in range(p)])
        #Devolve True se self for uma unidade no anel quociente.
        if R3(g).is_unit(): break
    #Privado, f é diferente de 0, logo invertivel em R/q, t>= 1
    f = randomR()
    #Polinómio h é a informação pública
    #h=g/(3f) no R/q, q é primo maior que 3, logo 3f é invertible R/q
    h = Rq(g)/(3*Rq(f))
    #Chave publica é h, basta fazer o encode
    pk = encodeRq(h)
    return pk,encodeZx(f) + encodeZx(R(lift(1/R3(g)))) + pk
#O remetente recebe uma chave publica como entrada
#E produz um texto cifrado e uma chave de sessao como saida.
def encapsulate(pk):
    #Decode a chave publica
    h = decodeRq(pk) 
    #Geral valor aleatorio pertencente a R
    r = randomR()
    #Calcular h*r
    hr = h * Rq(r)
    #nº inteiro entre -(q-1)/2 e (q-1)/2, para o multiplo mais
    #proximo de 3, produzindo assim c pertencente a R.
    m = Zx([-nicemod3(nicelift(hr[i])) for i in range(p)])  
    #Fqx.quotient(x^p-x-1) + h*r ..... GF(q)
    c = Rq(m) + hr  
    #Hash r, obtendo left half C (“Key confirmation”) e a right half K
    '''
    Codifica-se r como uma sequencia de bytes, add 1 a cada coeficiente,
    obtendo um elemento {0,1,2} codificando como 2 bits, e depois agrupar 4
    coeficientes adjacentes num byte, encodeZx.  
    '''
    fullkey = hash(encodeZx(r))
    #obtendo chave confirmação de 256-bit C e chave de sessao de 256-bit K  
    return fullkey[:32] + encoderoundedRq(c),fullkey[32:]

def decapsulate(cstr,sk):  
    #Processo de decode
    f,ginv,h = decodeZx(sk[:191]),decodeZx(sk[191:382]),decodeRq(sk[382:])  
    confirm,c = cstr[:32],decoderoundedRq(cstr[32:])  
    #Multiplica-se 3*f * c
    f3mgr = Rq(3*f) * c  
    #Ver cada coeficiente de 3fc em R/q como um inteiro
    #entre -(q-1)/2 e (q-1)/2 e depois menos modulo 3
    #Obtendo assim um polinomial e em R/3
    f3mgr = [nicelift(f3mgr[i]) for i in range(p)]
    #Multiplar por 1/g, em R/3
    r = R3(ginv) * R3(f3mgr)
    #Obtemos r - Lift e/g em R/3 para um pequeno polinimio r' pertencente a R
    r = Zx([nicemod3(lift(r[i])) for i in range(p)])  
    hr = h * Rq(r)  
    m = Zx([-nicemod3(nicelift(hr[i])) for i in range(p)])  
    #C', obtido pela r' como na encapsulaºão
    checkc = Rq(m) + hr  
    fullkey = hash(encodeZx(r)) 
    # Se r' is t-small, c' = c e C' = C, then output K'
    #Se não é falso
    if sum(r[i]==0 for i in range(p)) != p-2*t: 
        return False  
    if checkc != c: 
        return False  
    if fullkey[:32] != confirm: 
        return False  
    return fullkey[32:]


    
for keys in range(5):
    #Gerar chave publica e chave Secreta
    pk,sk = keygen()
    sk += pk#N sei se é necessario
    for ciphertexts in range(5):    
        c,k = encapsulate(pk)    
        assert decapsulate(c,sk) == k
    
return pk,sk
            
print(keygen())
print(randomR())
print (len(pk),'bytes in public key')
print (len(sk),'bytes in secret key')
print (len(c),'bytes in ciphertext')
print (len(k),'bytes in shared secret')

[1, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, -1, 0, -1, 1, -1, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, -1, 0, -1, -1, 1, 0, -1, -1, 0, 1, 0, -1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, -1, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, -1, 0, -1, 1, 0, -1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 1, -1, -1, 0, 0, 0, 0, 0, 0, -1, 1, 0, -1, -1, 0, 0, 0, 0, 1, -1, 0, 1, 1, 0, 0, -1, 1, 1, 0, 0, 0, 0, -1, -1, 0, -1, 1, -1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, -1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, -1, 0, 0, 0, 1, 0, -1, 0, 1, 0, 1, -1, 0, 0, 0, 0, 0, -1, 1, 1, 0, 0, -1, 0, 0, 1, 0, 0, 1, -1, 0, 1, 1, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 1, 0, 0, -1, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, -1, 0, 0, 1, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, -1, 0, -1, 0, 1, 0, -1, 0, 0, 0, 0, -1, -1, 0, 0, -1, 0, -

TypeError: Unicode-objects must be encoded before hashing